In [4]:
import cv2
import numpy as np
import os
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
import joblib
from matplotlib import pyplot as plt
%matplotlib inline

# If ImportError: libGL.so.1: cannot open shared object file: No such file or directory
# Run this in the terminal
# !apt update
# !apt install libgl1-mesa-glx



In [2]:
!pip install keras-facenet
!pip install mtcnn
!pip install joblib
!pip install sklearn
!pip install matplotlib

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 11.5 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 11.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [23]:
abs_path = '/scratch/ysolanki/SiameseFacebook/'
vid_path = abs_path + 'TestVideos/'
op_path = abs_path + 'output/'

# if not os.path.exists(op_path):
#     os.makedirs(op_path)
    
# 20170511-185253 - 128
# 20170512-110547 - 128 -- We trained on this 
embedder = FaceNet(key='20170512-110547')
print('FaceNet loaded')

model = joblib.load(abs_path + 'PakshalML/rfc_model.sav')
print('Classifier model loaded')

for file in os.listdir(vid_path):    
    if not file.endswith('.mp4'):
        continue
    vid = cv2.VideoCapture(vid_path + file)
    print("Loading " + file + "...")
    count = 0
    success = True

    real_count = 0
    fake_count = 0
    eq_count = 0
    writer = None

    while success:
        success, img = vid.read()

        if img is None:
            print('Frame %d does not exist' %count)
            continue

        count += 1

        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(op_path + file.split('.')[0] + '_op.avi', fourcc, 20, (img.shape[1], img.shape[0]), True)

        # This detects faces using MTCNN and returns 512/128 embeddings generated from FaceNet
        detections = embedder.extract(img)
        if not detections:
            writer.write(img)
    #         cv2.imwrite(op_path + "{:05d}.jpg".format(count), img)
            continue

    #   bounding_box = [x, y, width, height]
        bounding_box = detections[0]['box']
        x, y, width, height = bounding_box
        embeddings = detections[0]['embedding']
        embeddings = np.expand_dims(embeddings, axis=0)
        pred_proba = model.predict_proba(embeddings)

        if pred_proba[0][0] > pred_proba[0][1]:
            fake_count += 1
            text = 'Fake:' + str(pred_proba[0][0])
            rgb = (0, 0, 255)

        elif pred_proba[0][0] < pred_proba[0][1]:
            real_count += 1
            text = 'Real:' + str(pred_proba[0][1])
            rgb = (0,255,0)

        else:
            eq_count += 1
            text = ''
            rgb = (255,0,0)

#         text = 'Real:' + str(pred_proba[0][1]) + ' Fake:' + str(pred_proba[0][0])
        cv2.rectangle(img, (x, y), (x + width, y + height), rgb, 2)

    #         Move text down if it goes beyond image
        text_y = y - 15 if y - 15 > 15 else y + 15
        cv2.putText(img, text, (x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, rgb, 2)

    #     cv2.imwrite(op_path + "{:05d}.jpg".format(count), img)
        writer.write(img)

        if count % 100 == 0:
            print("Real : ", real_count, ", Fake : ", fake_count, ", Neither : ", eq_count)
    writer.release()

FaceNet loaded


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Classifier model loaded
Loading tc_v1.mp4...
Real :  0 , Fake :  100 , Neither :  0
Real :  4 , Fake :  313 , Neither :  3
Real :  7 , Fake :  407 , Neither :  3
Real :  7 , Fake :  507 , Neither :  3
Real :  9 , Fake :  589 , Neither :  4
Frame 1054 does not exist
Loading tc_v2.mp4...
Real :  0 , Fake :  98 , Neither :  0
Real :  0 , Fake :  197 , Neither :  1
Real :  0 , Fake :  294 , Neither :  1
Real :  0 , Fake :  488 , Neither :  1
Real :  0 , Fake :  588 , Neither :  1
Frame 614 does not exist
Loading nm_v1.mp4...
Real :  0 , Fake :  90 , Neither :  0
Real :  0 , Fake :  257 , Neither :  1
Real :  3 , Fake :  332 , Neither :  9
Real :  3 , Fake :  412 , Neither :  9
Real :  3 , Fake :  491 , Neither :  9
Real :  3 , Fake :  636 , Neither :  9
Real :  3 , Fake :  724 , Neither :  9
Frame 915 does not exist


In [15]:
vid_op = vid_path + 'output/tc_v1/'
files = os.listdir(op_path)
files.sort()
writer = None
for frame in files:
    frame = cv2.imread(op_path + frame)
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(vid_op + 'tc_v1_op.avi', fourcc, 20, (frame.shape[1], frame.shape[0]), True)
    writer.write(frame)
writer.release()

In [23]:
arr = []
a = [1,2,3]
b = [4,5,6]
c = [7,8,9]

arr.append(a)
print(arr)

[[1, 2, 3]]


In [9]:
rm -r ../TestVideos/output/tc_v1/frames

In [19]:
for file in os.listdir(vid_path):    
    if not file.endswith('.mp4'):
        continue
    print(file)

tc_v1.mp4
tc_v2.mp4
nm_v1.mp4
